In [13]:
# import pandas as pd
# import pulp as pl

# # Veriyi okuma
# df1 = pd.read_csv("set1_new.csv")
# df2 = pd.read_csv("set2.csv")

In [5]:
import pandas as pd
import pulp as pl
from collections import defaultdict

# Veri okuma
df1 = pd.read_csv("set1_new.csv")
df2 = pd.read_csv("set2.csv")

# Kullanıcının yaş ve cinsiyet bilgisi
age = 20
gender = "Kadın"

# Gerekli sınırlamaları alıyoruz
limits = df2[(df2["Cinsiyet"] == gender) & (df2["Yaş Grubu"].apply(lambda x: age in range(*map(int, x.split('-')))))].iloc[0]

# Kategori bilgisi
categories = {
    "main": range(1, 72),
    "soup": range(72, 100),
    "half_main": range(100, 150),
    "dessert_salad": range(150, 210),
}

price_conversion = {'a': 5, 'b': 4, 'c': 3, 'd': 2, 'e': 1}

# Her gün için kullanılan yemekleri saklamak için bir sözlük oluşturuyoruz
used_dishes = set()

# 4 hafta için 5 günlük menüler oluşturma
for week in range(1, 5):
    print(f"{week}. Hafta Menüleri\n")
    
    for day in range(1, 6):
        prob = pl.LpProblem(f"Menu_Optimization_{week}_{day}", pl.LpMaximize)
        
        available_dishes = list(set(df1['ID']) - used_dishes)
        dish_vars = pl.LpVariable.dicts("Dish", available_dishes, 0, 1, pl.LpBinary)

        # Amaç fonksiyonu
        prob += pl.lpSum([price_conversion[df1.set_index("ID").loc[i]["Fiyat"]] * dish_vars[i] for i in available_dishes])

        # Besin değeri kısıtlamaları
        for nutrient in ["Enerji", "Karbonhidrat", "Protein", "Yağ", "Lif"]:
            upper_limit = float(limits[nutrient].split('-')[1])
            prob += pl.lpSum([df1.set_index("ID").loc[i][nutrient] * dish_vars[i] for i in available_dishes]) <= upper_limit

        # Kategori kısıtlamaları
        for cat, rng in categories.items():
            prob += pl.lpSum([dish_vars[i] for i in available_dishes if i in rng]) == 1

        prob.solve(pl.PULP_CBC_CMD(msg=False))
        # prob.solve()

        # Çözümü yazdırma ve toplam besin değerlerini saklama
        print(f"{day}. Gün Menüsü:")
        total_nutrients = defaultdict(float)
        sorted_dishes = defaultdict(dict)
        
        for v in prob.variables():
            if v.varValue == 1:
                dish_id = int(v.name.split('_')[1])
                dish_info = df1.set_index("ID").loc[dish_id]
                
                # Yemek bu gün için kullanıldı, gelecekte bu gün için kullanılmayacak
                used_dishes.add(dish_id)
                
                # Yemeğin kategorisini bulma ve sıralamada saklama
                for cat, rng in categories.items():
                    if dish_id in rng:
                        sorted_dishes[cat] = {
                            "ID": dish_id,
                            "Yemek": dish_info['Yiyecek adı'],
                            "Fiyat": dish_info['Fiyat'],
                        }
                
                # Toplam besin değerlerini hesaplama
                for nutrient in ["Enerji", "Karbonhidrat", "Protein", "Yağ", "Lif"]:
                    total_nutrients[nutrient] += dish_info[nutrient]
        
        # Sıralanmış yemekleri yazdırma
        for cat in ["main", "soup", "half_main", "dessert_salad"]:
            dish = sorted_dishes[cat]
            print(f"{cat.upper()}: ID: {dish['ID']}, Yemek: {dish['Yemek']}, Fiyat: {dish['Fiyat']}")
        
        # Toplam besin değerlerini yazdırma
        print("Toplam Besin Değerleri:")
        for nutrient, value in total_nutrients.items():
            print(f"{nutrient}: {value}")

        print("---")
    print("\n")


1. Hafta Menüleri

1. Gün Menüsü:
MAIN: ID: 22, Yemek: Tas kebabı, Fiyat: a
SOUP: ID: 99, Yemek: Ispanaklı Lahana Çorbası, Fiyat: e
HALF_MAIN: ID: 110, Yemek: Kıymalı Tepsi Böreği, Fiyat: e
DESSERT_SALAD: ID: 208, Yemek: Salatalık turşusu, Fiyat: e
Toplam Besin Değerleri:
Enerji: 444.6
Karbonhidrat: 50.0
Protein: 20.9
Yağ: 17.4
Lif: 3.2
---
2. Gün Menüsü:
MAIN: ID: 18, Yemek: Patlıcan kebabı, Fiyat: a
SOUP: ID: 95, Yemek: Sütlü Arpa Şehriye Çorbası, Fiyat: e
HALF_MAIN: ID: 112, Yemek: Nohutlu Bulgur Pilavı, Fiyat: e
DESSERT_SALAD: ID: 184, Yemek: Karpuz, Fiyat: e
Toplam Besin Değerleri:
Enerji: 399.6
Karbonhidrat: 36.0
Protein: 17.5
Yağ: 20.500000000000004
Lif: 4.8
---
3. Gün Menüsü:
MAIN: ID: 3, Yemek: Rosto et(patatesli), Fiyat: a
SOUP: ID: 76, Yemek: Domatesli Pirinç Çorbası, Fiyat: e
HALF_MAIN: ID: 115, Yemek: Buhara Pilavı, Fiyat: e
DESSERT_SALAD: ID: 157, Yemek: Marul Salatası, Fiyat: e
Toplam Besin Değerleri:
Enerji: 434.9
Karbonhidrat: 39.099999999999994
Protein: 22.6
Yağ: 20.4

In [8]:
import pandas as pd
import pulp as pl
from collections import defaultdict

# Veri okuma
df1 = pd.read_csv("set1_new.csv")
df2 = pd.read_csv("set2.csv")

# Kullanıcının yaş ve cinsiyet bilgisi
age = 20
gender = "Kadın"

# Gerekli sınırlamaları alıyoruz
limits = df2[(df2["Cinsiyet"] == gender) & (df2["Yaş Grubu"].apply(lambda x: age in range(*map(int, x.split('-')))))].iloc[0]

# Kategori bilgisi
categories = {
    "main": range(1, 72),
    "soup": range(72, 100),
    "half_main": range(100, 150),
    "dessert_salad": range(150, 210),
}

price_conversion = {'a': 5, 'b': 4, 'c': 3, 'd': 2, 'e': 1}

# Her gün için kullanılan yemekleri saklamak için bir sözlük oluşturuyoruz
used_dishes = set()

# 4 hafta için 5 günlük menüler oluşturma
for week in range(1, 5):
    print(f"{week}. Hafta Menüleri\n")
    
    for day in range(1, 6):
        prob = pl.LpProblem(f"Menu_Optimization_{week}_{day}", pl.LpMaximize)
        
        available_dishes = list(set(df1['ID']) - used_dishes)
        dish_vars = pl.LpVariable.dicts("Dish", available_dishes, 0, 1, pl.LpBinary)

        # Amaç fonksiyonu
        prob += pl.lpSum([price_conversion[df1.set_index("ID").loc[i]["Fiyat"]] * dish_vars[i] for i in available_dishes])

        # Besin değeri kısıtlamaları
        for nutrient in ["Enerji", "Karbonhidrat", "Protein", "Yağ", "Lif"]:
            lower_limit = float(limits[nutrient].split('-')[0])
            upper_limit = float(limits[nutrient].split('-')[1])
            
            prob += pl.LpConstraint(
                e=pl.lpSum([df1.set_index("ID").loc[i][nutrient] * dish_vars[i] for i in available_dishes]),
                sense=pl.LpConstraintGE,
                name=f"{nutrient}_lower_bound",
                rhs=lower_limit
            )
            
            prob += pl.LpConstraint(
                e=pl.lpSum([df1.set_index("ID").loc[i][nutrient] * dish_vars[i] for i in available_dishes]),
                sense=pl.LpConstraintLE,
                name=f"{nutrient}_upper_bound",
                rhs=upper_limit
            )

        # Kategori kısıtlamaları
        for cat, rng in categories.items():
            prob += pl.lpSum([dish_vars[i] for i in available_dishes if i in rng]) == 1

        prob.solve(pl.PULP_CBC_CMD(msg=False))

        # Çözümü yazdırma ve toplam besin değerlerini saklama
        print(f"{day}. Gün Menüsü:")
        total_nutrients = defaultdict(float)
        sorted_dishes = defaultdict(dict)
        
        for v in prob.variables():
            if v.varValue == 1:
                dish_id = int(v.name.split('_')[1])
                dish_info = df1.set_index("ID").loc[dish_id]
                
                used_dishes.add(dish_id)
                
                for cat, rng in categories.items():
                    if dish_id in rng:
                        sorted_dishes[cat] = {
                            "ID": dish_id,
                            "Yemek": dish_info['Yiyecek adı'],
                            "Fiyat": dish_info['Fiyat'],
                        }
                
                for nutrient in ["Enerji", "Karbonhidrat", "Protein", "Yağ", "Lif"]:
                    total_nutrients[nutrient] += dish_info[nutrient]
        
        for cat in ["main", "soup", "half_main", "dessert_salad"]:
            dish = sorted_dishes.get(cat)
            if dish:
                print(f"{cat.upper()}: ID: {dish['ID']}, Yemek: {dish['Yemek']}, Fiyat: {dish['Fiyat']}")
            else:
                print(f"{cat.upper()}: Bu kategoride yemek seçilememiştir.")
            # print(f"{cat.upper()}: ID: {dish['ID']}, Yemek: {dish['Yemek']}, Fiyat: {dish['Fiyat']}")
        
        print("Toplam Besin Değerleri:")
        for nutrient, value in total_nutrients.items():
            print(f"{nutrient}: {value}")

        print("---")
    print("\n")


1. Hafta Menüleri

1. Gün Menüsü:
MAIN: ID: 8, Yemek: İzmir Köfte, Fiyat: b
SOUP: Bu kategoride yemek seçilememiştir.
HALF_MAIN: ID: 149, Yemek: Kabak Kalye, Fiyat: e
DESSERT_SALAD: ID: 209, Yemek: Lahana Turşusu, Fiyat: e
Toplam Besin Değerleri:
Enerji: 5817.2
Karbonhidrat: 466.7
Protein: 320.3
Yağ: 289.20000000000005
Lif: 99.1
---
2. Gün Menüsü:
MAIN: ID: 69, Yemek: Kağıt Kebabı, Fiyat: a
SOUP: ID: 92, Yemek: Nohutlu Pirinç Çorbası, Fiyat: e
HALF_MAIN: ID: 141, Yemek: Erişte, Fiyat: e
DESSERT_SALAD: ID: 168, Yemek: Kuru Kayısı Hoşafı, Fiyat: e
Toplam Besin Değerleri:
Enerji: 4498.3
Karbonhidrat: 399.4
Protein: 264.5
Yağ: 200.7
Lif: 56.0
---
3. Gün Menüsü:
MAIN: ID: 62, Yemek: Kıymalı Yaprak Sarma, Fiyat: c
SOUP: ID: 81, Yemek: Patates Çorbası, Fiyat: e
HALF_MAIN: ID: 128, Yemek: Zeytinyağlı Kereviz, Fiyat: e
DESSERT_SALAD: ID: 208, Yemek: Salatalık turşusu, Fiyat: e
Toplam Besin Değerleri:
Enerji: 2282.2999999999997
Karbonhidrat: 226.4
Protein: 64.29999999999998
Yağ: 117.499999999999